In [1]:
import numpy as np
import random

In [2]:
def GrayСode(BinLen=32):
    return np.random.randint(2, size=BinLen)

In [3]:
def GrayToBinar(bit_string):
    """
    Правило перевода из кода Грея в позиционный код: каждый разряд в позиционном
    коде равен сумме по модулю два этого и всех старших разрядов в коде Грея
    """
    return [sum(bit_string[:i]) % 2 for i in range(len(bit_string),0,-1)][::-1]

In [4]:
def BinaryToInt(bit_string):
    """
    """
    return sum([int(x) * 2 ** y for x, y in zip(list(bit_string), range(len(bit_string) - 1, -1, -1))])

In [5]:
class Decode:
    """
    Полный цикл декодирования: Грей -> двоичный -> целое -> нецелое
    """
    def __init__(self,a,b,bit_string):
        self.A = a
        self.B = b
        self.BitString = bit_string
        self.k = 2**len(bit_string) - 1 
        self.h = (b - a)/(self.k - 1)
    
    def decode(self):
        binary_srting = GrayToBinar(self.BitString)
        #print('binary: ',binary_srting)
        integer = BinaryToInt(binary_srting)
        #print('integer: ',integer)
        if integer == 0:
            return self.A
        elif integer == self.k:
            return self.B
        else:
            return ((self.A + (integer - 1)*self.h + (self.A+integer*self.h)))/2

In [6]:
def SinglePointCrossing(parent_one,parent_two):
    """
    По хорошему нужен более качественный генератор случайных чисел
    """
    point = np.random.randint(min(len(parent_one),len(parent_two)) - 1)
    return np.hstack([parent_one[:point],parent_two[point:]])

In [27]:
def Mutation(individual,probability = 0.5):
    """
    вероятность поменять ген
    """
    for i in range(len(individual)):
        if np.random.rand() > probability:
            individual[i] = (individual[i] + 1) % 2
    return individual

In [8]:
def f(x):
    return -(x**2)

In [9]:
a = -5
b = 5

In [57]:
IndNum = 10
population = [GrayСode(8) for i in range(IndNum)] # Это вот инициализация 

In [58]:
# Функция приспособленности
def MyFunc(x):
    return f(Decode(a,b,x).decode())

In [84]:
ChildrenNum = 5
MutationNum = 5
TopNum = 10

IndNum = 20
population = [GrayСode(32) for i in range(IndNum)] # Это вот инициализация 

for i in range(10):
    #print('=== STEP {} ==='.format(i))
    population = sorted(population,key=MyFunc,reverse=True) # reversed = True, когда ищем максимум
    # Хотим взять n самых крутых родителей
    children = []
    for j in range(ChildrenNum):
        children.append(SinglePointCrossing(random.sample(population[:TopNum], 1)[0],random.sample(population[:TopNum], 1)[0]))
    # Продумать этот момент получше
    for j in range(MutationNum):
        children.append(Mutation(random.sample(population, 1)[0]))
    population = np.vstack([population,children])
    population = sorted(population,key=MyFunc,reverse=True)[:10] # оставляем 10 лучших особей

In [85]:
for i in population:
    print(Decode(a,b,i).decode())

0.12102795886855056
0.1210479590208493
0.12104796134915574
0.12107562628624713
0.12107564258439307
0.12309264746638604
0.16215514748457593
0.1649732713890142
0.1649733552080459
0.1649733552080459


In [112]:
def InitFunction(BinLen,IndNum):
    return [GrayСode(BinLen) for i in range(IndNum)]

In [113]:
class GeneticAlgorithm:
    def __init__(self,BinLen,IndNum,MutationNum,TopNum,ChildrenNum,Steps,InitFunction,AdaptabilityFunction,MutationFunction,CrossingFunction):
        self.BinLen = BinLen
        self.IndNum = IndNum
        self.ChildrenNum = ChildrenNum
        self.MutationNum = MutationNum
        self.TopNum = TopNum
        self.Steps = Steps
        self.InitFunction = InitFunction
        self.AdaptabilityFunction = AdaptabilityFunction 
        self.MutationFunction = MutationFunction
        self.CrossingFunction = CrossingFunction
    
    def run(self):
        self.population = self.InitFunction(self.BinLen,self.IndNum)
        for i in range(self.Steps):
            self.population = sorted(self.population,key=self.AdaptabilityFunction,reverse=True)
            children = []
            for j in range(self.ChildrenNum):
                children.append(self.CrossingFunction(
                    random.sample(self.population[:self.TopNum], 1)[0],
                    random.sample(self.population[:self.TopNum], 1)[0]))
            for j in range(self.MutationNum):
                children.append(self.MutationFunction(random.sample(self.population, 1)[0]))
            self.population = np.vstack([self.population,children])
            self.population = sorted(self.population,key=self.AdaptabilityFunction,reverse=True)[:10]
        return(self.population)

In [114]:
testGen = GeneticAlgorithm(
    BinLen=32, IndNum=20,MutationNum=10,TopNum=10,ChildrenNum=10,Steps=100, InitFunction = InitFunction,
    AdaptabilityFunction=MyFunc,MutationFunction=Mutation, CrossingFunction=SinglePointCrossing)

In [115]:
answers = testGen.run()

In [116]:
for i in answers:
    print(Decode(-5,5,i).decode())

-0.0029400934013255053
-0.0029400934013255053
-0.0029400934013255053
-0.0029400934013255053
-0.0029400934013255053
-0.0029400934013255053
-0.0029400934013255053
-0.0029400934013255053
-0.0029400934013255053
-0.0029400934013255053


In [ ]:
def f(x):
    """
    Попробуем написать вот так
    """
    return -(x[0]**2 + x[1])